In [ ]:
from stock.tw_stock import tw_stock
import datetime
import pandas as pd
tw = tw_stock()
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2022, 3, 1)


In [10]:

# 一、單支股票週線策略回測
def breakout_weekly_strategy(weekly_df, stock_id, high_window=50, low_window=40):
    result = []
    position = False
    entry_price = None
    entry_date = None

    for i in range(max(high_window, low_window), len(weekly_df)):
        window_high = weekly_df['high'].iloc[i - high_window:i].max()
        window_low = weekly_df['low'].iloc[i - low_window:i].min()

        week_high = weekly_df['high'].iloc[i]
        week_low = weekly_df['low'].iloc[i]
        week_close = weekly_df['close'].iloc[i]
        week_date = weekly_df.index[i]

        if not position and week_high > window_high:
            position = True
            entry_price = week_close
            entry_date = week_date

        elif position and week_low < window_low:
            return_pct = (week_close - entry_price) / entry_price
            result.append({
                '股票': stock_id,
                '進場日': entry_date,
                '出場日': week_date,
                '進場價': round(entry_price, 2),
                '出場價': round(week_close, 2),
                '報酬率': round(return_pct * 100, 2),
                '持有週數': (week_date - entry_date).days // 7
            })
            position = False

    return pd.DataFrame(result)

# 二、多股票週線回測
def run_breakout_backtest(stock_ids, tw, start, end):
    all_result = []

    for stock_id in stock_ids:
        try:
            df = tw.get_stock(stock_id, start, end)
            df = df.set_index("date").sort_index()

            # 日轉週
            weekly = pd.DataFrame()
            weekly['open'] = df['open'].resample('W-FRI').first()
            weekly['high'] = df['high'].resample('W-FRI').max()
            weekly['low'] = df['low'].resample('W-FRI').min()
            weekly['close'] = df['close'].resample('W-FRI').last()
            weekly['volume'] = df['volume'].resample('W-FRI').sum()
            weekly.dropna(inplace=True)

            if len(weekly) < 60:
                continue

            res = breakout_weekly_strategy(weekly, stock_id)
            if not res.empty:
                all_result.append(res)

        except Exception as e:
            print(f"{stock_id} 發生錯誤: {e}")
            continue

    return pd.concat(all_result) if all_result else pd.DataFrame()

# 三、執行策略
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2022, 3, 1)
tw = tw_stock()

# 只跟蹤前 100 種股票為練習用
all_ids = ['2330', '2303', '0050', '2603', '2317', '2882', '2412', '1301', '2002', '1101']
df_result = run_breakout_backtest(all_ids, tw, start, end)
df_result.reset_index(drop=True, inplace=True)
#df_result.to_csv("weekly_breakout_result.csv", index=False)
df_result.head()


,股票,進場日,出場日,進場價,出場價,報酬率,持有週數
0,2330,2016-03-18,2018-06-29,159.50,216.50,35.74,119
1,2330,2018-08-31,2019-01-04,256.00,208.00,-18.75,18
2,2303,2017-06-30,2018-10-12,14.75,13.65,-7.46,67
3,0050,2016-07-22,2018-10-12,69.30,80.05,15.51,116
4,0050,2019-10-04,2020-03-13,85.15,81.00,-4.87,23
